<a href="https://colab.research.google.com/github/jakobatgithub/unreverb/blob/main/ClassificationOfReverb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_io pyyaml h5py

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
import os

sample_rate = 2**14

# function to delete all content in a folder
def delete_folder_contents(folder):
  for filename in os.listdir(folder):
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))  

# load data
path = '/content/drive/My Drive/dsr_project/data/but-czas_v1.0/'
save_path_M = path + 'datasets_randIR_M/'
save_path_F = path + 'datasets_randIR_F/'

dataset_filenames_M = glob.glob(save_path_M + 'tf_IR_*')
dataset_filenames_F = glob.glob(save_path_F + 'tf_IR_*')
dataset_filenames = dataset_filenames_M + dataset_filenames_F
#labels = [int(filename.split('_')[-1]) for filename in dataset_filenames]
#print(f"labels: {labels}")

datasets = []
for filename in dataset_filenames:
  # load the tensorflow dataset
  datasets.append(tf.data.experimental.load(filename))

dataset = datasets[0]
#for i in range(1, len(datasets)):
for i in range(1, 10):
  dataset = dataset.concatenate(datasets[i])

# determine size of the dataset
dataset_size = sum(1 for _ in dataset)
print(f"dataset_size: {dataset_size}")

In [ ]:
def lambda_1(features, labels, targets):
  return (features, labels)

def lambda_2(features, labels, targets):
  return (features, targets)

def lambda_3(features, labels, targets):
  return labels

# obtain the datasets containing only labels or targets
label_dataset = dataset.map(lambda features, labels, targets: lambda_1(features, labels, targets))
target_dataset = dataset.map(lambda features, labels, targets: lambda_2(features, labels, targets))

# obtain all labels
all_labels = dataset.map(lambda features, labels, targets: lambda_3(features, labels, targets))

# transform all_labels dataset to numpy array
all_labels_np = []
for label in all_labels:
  all_labels_np.append(tfds.as_numpy(label))

all_labels_np = np.array(all_labels_np)

# determine unique labels
unique_labels, counts = np.unique(all_labels_np, return_counts=True)
print(f"unique_labels: {unique_labels}")
#print(counts)

# determine number of unique labels
nr_of_unique_labels = unique_labels.shape[0]
print(f"nr_of_unique_labels: {nr_of_unique_labels}")

# shuffle the dataset before splitting in train and validate!
label_dataset = label_dataset.shuffle(dataset_size)

# split dataset in train and validate
train_fraction = 0.8
validate_dataset_size = int(dataset_size * (1.0-train_fraction)) # 20 percent of dataset_size
train_dataset = label_dataset.skip(validate_dataset_size)
validate_dataset = label_dataset.take(validate_dataset_size)

total_dataset = dataset

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import librosa
import numpy as np

for audio, label  in train_dataset.shuffle(dataset_size).take(1):
  audio = audio.numpy().astype("float32")
  label = label.numpy()

  plt.plot(audio)
  plt.title(f"Label: {label}")
  plt.show()
  plt.close()

  display(Audio(audio, rate=sample_rate))

  mel = librosa.feature.melspectrogram(
      y=audio, n_mels=128, hop_length=64, sr=sample_rate, fmax=2000
  )

  mel /= np.max(mel)
  plt.imshow(mel[::-1, :], cmap="inferno")

In [ ]:
from scipy.signal.spectral import spectrogram
import tensorflow as tf
import tensorflow_io as tfio
from keras.preprocessing.sequence import pad_sequences

def preprocess(sample, target):
  audio = tf.cast(sample, tf.float32)
  label = target
  #print(audio)
  #print(label)
  spectrogram = tfio.audio.spectrogram(audio, nfft=1024, window=1024, stride=64)
  spectrogram = tfio.audio.melscale(spectrogram, rate=sample_rate, mels=128, fmin=0, fmax=2000)
  spectrogram /= tf.math.reduce_max(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=-1)

  spectrogram = tf.image.resize(spectrogram, (128, 128))
  spectrogram = tf.transpose(spectrogram, perm=(1, 0, 2))
  spectrogram = spectrogram[::-1, :, :]

  return spectrogram, label


dataset_check = train_dataset.map(lambda sample, target: preprocess(sample, target))

for x, y in dataset_check.shuffle(dataset_size).take(4):
  plt.imshow(x[:,:,0], cmap="inferno")
  plt.title(f"{x.shape}, label: {y.numpy()}")
  plt.show()
  plt.close()

In [ ]:
dataset_train = train_dataset.map(lambda sample, target: preprocess(sample, target))
dataset_train = dataset_train.cache()
dataset_train = dataset_train.shuffle(dataset_size)
dataset_train = dataset_train.batch(32)

dataset_validate = validate_dataset.map(lambda sample, target: preprocess(sample, target))
dataset_validate = dataset_validate.cache()
dataset_validate = dataset_validate.batch(32)

In [ ]:
from tensorflow.keras import models, layers

my_input_shape = (128, 128, 1)

model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(8, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(nr_of_unique_labels, activation="softmax"))
model.summary()

model.compile(
    optimizer="adam",
    #loss=tf.keras.losses.MeanSquaredError(),
    #loss="binary_crossentropy",
    loss="sparse_categorical_crossentropy",
    #loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    dataset_train,
    epochs=400,
    validation_data=dataset_validate
)

model.save(save_path_F + 'model_randIR.h5')

In [ ]:
model = tf.keras.models.load_model(save_path_F + 'model_randIR.h5')

In [ ]:
history = model.fit(
    dataset_train,
    epochs=100,
    validation_data=dataset_validate
)

In [ ]:
model.save(save_path_F + 'model_randIR.h5')

In [ ]:
def render_history(history):
    plt.plot(history["loss"], label="loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend()
    plt.title("Our losses")
    plt.show()
    plt.close()

    plt.plot(history["accuracy"], label="accuracy")
    plt.plot(history["val_accuracy"], label="val_accuracy")
    plt.legend()
    plt.title("Our accuracies")
    plt.show()
    plt.close()


def compare_histories():
    for training_name, history in history_list.items():
        plt.plot(history["val_accuracy"], label=training_name)
    plt.legend()
    plt.title("Comparison of val_accuracy")
    plt.show()
    plt.close()

render_history(history.history)